In [ ]:
import pandas as pd
import numpy as np
import nltk
from textblob import TextBlob
from math import log

In [ ]:
dados = pd.read_csv('../data/estadao_noticias_eleicao.csv')
dados = dados.replace(np.nan, '', regex=True)

### Visão geral dos dados

In [ ]:
dados.head()

In [ ]:
print("Os dados tem %d observações e %d colunas" % dados.shape)

In [ ]:
def calculaTF(documento, termo):
    return len(list(filter((lambda x: x.lower() == termo), documento)))

In [ ]:
def calcula_indices_invertidos(data):
    indices = {}
    for index, row in data.iterrows():
        print(index, ",", row)
        titulo = TextBlob(row["titulo"]).words
        subTitulo = TextBlob(row["subTitulo"]).words
        conteudo = TextBlob(row["conteudo"]).words
        
        texto = titulo + subTitulo + conteudo
        
        for palavra in texto:
            palavra_low = palavra.lower()
            if(palavra_low in indices):
                temp = indices[palavra_low]
                if(row["idNoticia"] not in temp):
                    temp[row["idNoticia"]] = calculaTF(texto, palavra_low)
                    indices[palavra_low] = temp
            else:
                indices[palavra_low] = {row["idNoticia"]: calculaTF(texto, palavra_low)}
    return(indices)

indices_invertidos = calcula_indices_invertidos(dados)

In [ ]:
n_documentos = dados.shape[0]

def idf(palavra):
    palavra = palavra.lower()
    k = len(indices_invertidos[palavra])
    idf = log((n_documentos+1)/k)
    
    return(idf)

In [ ]:
#representação binária;
#TF;
#TF-IDF; 
# BM25

In [ ]:
def calcula_tf(query, documento):
    total = 0;
    for palavra in query:
        if(palavra in indices_invertidos):
            temp = indices_invertidos[palavra]
            if(documento in temp):
                total += temp[documento]
    return(total)
def calcula_

In [ ]:
def busca_documentos(query):
    documentos = set()
    for palavra in query:
        if(palavra in indices_invertidos):
            documentos = documentos | set(indices_invertidos[palavra])
    return(list(documentos))

In [90]:
def ranking_tf(query):
    docs = busca_documentos(query)
    tf_docs = []
    for doc in docs:
        tf_docs.append(calcula_tf(query, doc))
    docs_ordenado = [docs[i] for i in np.argsort(tf_docs)[::-1]]
    return(docs_ordenado[0:5])

ranking_tf(["segundo", "turno"])

[2744, 7, 7672, 2112, 2388]

In [ ]:
def ranking_idf(query):
    docs = busca_documentos(query)
    idf_docs = []
    for doc in docs:
        